IMDB

In [ ]:
from fastai import *
from fastai.text import *

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path.ls()

[PosixPath('/root/.fastai/data/imdb_sample/texts.csv')]

In [ ]:
df = pd.read_csv(path/"texts.csv")

In [ ]:
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [ ]:
len(df)

1000

In [ ]:
df['text'][0]

"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!"

Tokenizataion

In [ ]:
data_lm = TextDataBunch.from_csv(path, 'texts.csv')

In [ ]:
data_lm.save()

In [ ]:
data = load_data(path)
data.show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , xxunk bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj xxunk , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first stealth games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - rounded gaming experience in general . xxmaj with graphics",positive
"xxbos \n \n i 'm sure things did n't exactly go the same way in the real life of xxmaj homer xxmaj hickam as they did in the film adaptation of his book , xxmaj rocket xxmaj boys , but the movie "" xxmaj october xxmaj sky "" ( an xxunk of the book 's title ) is good enough to stand alone . i have not read xxmaj",positive


Numericalization

In [ ]:
#the big list of tokens is also called as vocabulary
data.vocab.itos[:500]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 '.',
 ',',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'i',
 'this',
 'that',
 '"',
 "'s",
 '-',
 'was',
 '\n \n ',
 'for',
 'movie',
 'as',
 'with',
 'but',
 'film',
 'you',
 ')',
 '(',
 'on',
 "n't",
 'are',
 'not',
 'his',
 'he',
 'have',
 'be',
 'one',
 'they',
 'all',
 'at',
 'by',
 '!',
 'like',
 'an',
 'from',
 'who',
 'so',
 'there',
 'if',
 'or',
 'about',
 'just',
 'out',
 'some',
 'do',
 'what',
 "'",
 'her',
 'has',
 '?',
 'more',
 'good',
 'very',
 'when',
 'no',
 'up',
 'were',
 'would',
 'their',
 'really',
 'only',
 'my',
 'time',
 'she',
 'even',
 'had',
 'can',
 'story',
 'which',
 'see',
 'did',
 ':',
 'me',
 '...',
 'we',
 'well',
 'will',
 'does',
 'than',
 'get',
 'into',
 'also',
 'bad',
 'much',
 'could',
 'other',
 'first',
 'people',
 'because',
 'any',
 'then',
 'great',
 'most',
 'make',
 'how',
 'been',
 'two',
 'them',
 'made',
 'its',
 'him',
 'too',
 

In [ ]:
data.train_ds

LabelList (799 items)
x: TextList
xxbos good movie , good music , good background and an acceptable plot . but the main point again as his movies tend to be , the man is the best actor in xxunk and can turn dust into gold . xxunk xxunk . this may be his second best performance after xxunk ( others may disagree ) . although other movies are not far behind . one man that will never ever disappoint you . 
 
  good movie although i think xxunk was a luxury this movie could have done without . you can see in his movies , others try very hard to reach his heights and act out of their skins . but this man is really something xxunk . 
 
  the movie is cool , the music and direction is excellent plot a bit thin but the screen play and dialog again very good . a must watch .,xxbos xxmaj come on . xxmaj the new twist is nearly ok , but from xxunk the xxmaj elm xxmaj street children xxmaj freddy is just killing people now . xxmaj more of the same : xxmaj special effects with no actual character de

In [ ]:
data.train_ds[0][0].data[:10]

array([   2,   69,   28,   11,   69,  301,   11,   69, 1146,   12])

Data Block API

In [ ]:
data = (TextList.from_csv(path, 'texts.csv', cols='text')
                .split_from_df(col=2)
                .label_from_df(cols=0)
                .databunch())

Now let's grab the full data set for what follows!
Now let's grab the whole data set which has:
25,000 reviews in training
25,000 interviews in validation
50,000 unsupervised movie reviews (50,000 movie reviews that haven't been scored at all)

In [ ]:
path = untar_data(URLs.IMDB)
path.ls()

[PosixPath('/root/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/root/.fastai/data/imdb/unsup'),
 PosixPath('/root/.fastai/data/imdb/test'),
 PosixPath('/root/.fastai/data/imdb/tmp_clas'),
 PosixPath('/root/.fastai/data/imdb/train'),
 PosixPath('/root/.fastai/data/imdb/tmp_lm'),
 PosixPath('/root/.fastai/data/imdb/README')]

In [ ]:
(path/'train').ls()

[PosixPath('/root/.fastai/data/imdb/train/pos'),
 PosixPath('/root/.fastai/data/imdb/train/neg'),
 PosixPath('/root/.fastai/data/imdb/train/unsupBow.feat'),
 PosixPath('/root/.fastai/data/imdb/train/labeledBow.feat')]

Language Model : We are going to use pretrained and we  gonna start fine-tuning our IMDB language model.


In [ ]:
bs = 48

In [ ]:
data_lm = (TextList.from_folder(path)
           #Inputs: all the text files in path
            .filter_by_folder(include=['train', 'test'])
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.1)
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_for_lm()
           #We want to do a language model so we label accordingly
            .databunch(bs=bs))


In [ ]:
data_lm.save('data_ready')

In [ ]:
data_lm.show_batch()

idx,text
0,"xxup below , but the one scene that made an extremely deep impression on me was xxmaj sterling xxmaj holloway 's death scene , which several other commenters have mentioned here . i have n't gotten to that scene yet on this viewing , but i can vouch for what other comments have said : once you see xxmaj sterling xxmaj holloway 's death scene in this movie , you"
1,"cast . xxmaj it will touch the deepest reaches of your emotions , for those who belief that a love that risks nothing , is worth nothing . xxmaj the film starts with introducing us to the characters that on the surface seem to have everything , later we find that beneath the surfaces of each character they are looking for a journey of self discovery . xxmaj we learn"
2,"recall , it was quite humorous . xxmaj would love to find it on tape . xxbos i 'm a fan of xxmaj zhang xxmaj yimou and finally found this xxup dvd title from the shelves of a xxmaj xxunk bookstore after a long search at many places . \n \n xxmaj this is a huge departure from previous xxmaj zhang xxmaj yimou work , esp in terms of"
3,"i 'll take a whole weekend and watch all the installments back to back - and am sad when the last episode rolls to an end , because i find myself wanting to continue watching the story of the lives of these characters . \n \n i can not recommend this mini - series more highly . xxbos i saw this movie yesterday and ca n't stop thinking about"
4,". xxmaj if the good guys and bad guys are too obviously pronounced for a serious film , and yes xxmaj sam xxmaj waterston 's mustachioed , fur - clad villain is comic - opera ( and not in the multi - leveled manner of xxmaj bill xxmaj the xxmaj butcher from ' xxmaj gangs of xxmaj new xxmaj york ' ) , and yes , the townsfolk do seem"


In [ ]:
learn = learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.lr_find()

epoch,train_loss,valid_loss,accuracy,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: ignored

In [ ]:
learn.recorder.plot(skip_end =15)

In [ ]:
learn.fit_one_cycle(1,1e-2,moms =(0.8,0.7))

In [ ]:
learn.save("fit_head")

In [ ]:
learn.load('fit_head')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10,1e-3,moms = (0.8,0.7))

In [ ]:
learn.save_encoder("fine-tuned")